In [14]:
from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()
# app_context.pop()

In [15]:
from app.models1 import *
from app.models import *
import random
from datetime import datetime


In [16]:
c = Channel.query.filter_by(id=1).first()
print(c)

None


各个机场代码如下：其中山东临沂机场没有代码，而南宁吴圩国际机场和上海浦东国际机场有两个，在存数据库时用其中一个
```
ZYTL	大连周水子国际机场
ZGGG	广州白云国际机场
ZSHC	杭州萧山国际机场
WHJC	武汉机场
ZGSD	珠港机场
JYCS	揭阳潮汕机场
nng	南宁吴圩国际机场
NNG	南宁吴圩国际机场
#	山东临沂机场
ZSPD	上海浦东国际机场
ZSPDbs	上海浦东国际机场
ZGSZ	深圳机场
ZBTJ	天津滨海国际机场
YW	浙江省义乌机场
ZHCC	郑州新郑国际机场
ZUCK	重庆机场集团公司飞行区管理部
CSX	长沙黄花国际机场
CZX	常州国际机场有限公司
XZ	徐州机场
ZBAD	北京大兴国际机场
ZSNT	南通兴东国际机场
```


In [17]:
code1 = ['ZYTL','ZGGG','ZSHC','WHJC','ZGSD','JYCS','NNG','ZSPD',
        'ZGSZ','ZBTJ','YW','ZHCC','ZUCK','CSX','CZX','XZ','ZBAD','ZSNT']
name1 = ['大连周水子国际机场','广州白云国际机场','杭州萧山国际机场','武汉机场','珠港机场','揭阳潮汕机场',
        '南宁吴圩国际机场','上海浦东国际机场','深圳机场',
        '天津滨海国际机场','浙江省义乌机场','郑州新郑国际机场','重庆机场集团公司飞行区管理部',
        '长沙黄花国际机场','常州国际机场有限公司','徐州机场','北京大兴国际机场','南通兴东国际机场']
len(name1)

18

In [18]:
# 加入各个机场（渠道）信息
for i in range(18):
    c = Channel.query.filter_by(code=code1[i]).first()
    if not c :
        c = Channel(code1[i],name1[i])
        db.session.add(c)
db.session.commit()

In [19]:
# 创建用户信息：
for i in range(18):
    u = User.query.filter_by(username=name1[i]).first()
    if not u :
        u = User()
        u.username = name1[i]
        u.set_password(code1[i])
        
        c = Channel.query.filter_by(code=code1[i]).first()
        u.channel =c 
        db.session.add(u)
db.session.commit()

In [20]:
User.query.all()

[<User 大连周水子国际机场>,
 <User 广州白云国际机场>,
 <User 杭州萧山国际机场>,
 <User 武汉机场>,
 <User 珠港机场>,
 <User 揭阳潮汕机场>,
 <User 南宁吴圩国际机场>,
 <User 上海浦东国际机场>,
 <User 深圳机场>,
 <User 天津滨海国际机场>,
 <User 浙江省义乌机场>,
 <User 郑州新郑国际机场>,
 <User 重庆机场集团公司飞行区管理部>,
 <User 长沙黄花国际机场>,
 <User 常州国际机场有限公司>,
 <User 徐州机场>,
 <User 北京大兴国际机场>,
 <User 南通兴东国际机场>]

In [21]:
# 由于测序数据的code并不统一，有些机场有两个编码：
code = ['ZYTL','ZGGG','ZSHC','WHJC','ZGSD','JYCS','nng','NNG','ZSPD','ZSPDbs',
        'ZGSZ','ZBTJ','YW','ZHCC','ZUCK','CSX','CZX','XZ','ZBAD','ZSNT']
name = ['大连周水子国际机场','广州白云国际机场','杭州萧山国际机场','武汉机场','珠港机场','揭阳潮汕机场',
        '南宁吴圩国际机场','南宁吴圩国际机场','上海浦东国际机场','上海浦东国际机场','深圳机场',
        '天津滨海国际机场','浙江省义乌机场','郑州新郑国际机场','重庆机场集团公司飞行区管理部',
        '长沙黄花国际机场','常州国际机场有限公司','徐州机场','北京大兴国际机场','南通兴东国际机场']
len(name)

20

In [22]:
airport_dict = dict(zip(code,name))
print(airport_dict)

{'ZYTL': '大连周水子国际机场', 'ZGGG': '广州白云国际机场', 'ZSHC': '杭州萧山国际机场', 'WHJC': '武汉机场', 'ZGSD': '珠港机场', 'JYCS': '揭阳潮汕机场', 'nng': '南宁吴圩国际机场', 'NNG': '南宁吴圩国际机场', 'ZSPD': '上海浦东国际机场', 'ZSPDbs': '上海浦东国际机场', 'ZGSZ': '深圳机场', 'ZBTJ': '天津滨海国际机场', 'YW': '浙江省义乌机场', 'ZHCC': '郑州新郑国际机场', 'ZUCK': '重庆机场集团公司飞行区管理部', 'CSX': '长沙黄花国际机场', 'CZX': '常州国际机场有限公司', 'XZ': '徐州机场', 'ZBAD': '北京大兴国际机场', 'ZSNT': '南通兴东国际机场'}


In [27]:
# old_date和old_code的目的是为了判断是否是同一个订单同一个机场
old_code = ''
old_date = ''
old_order  = None
i = 0
with open('airport.csv') as fh,open('cannot_find_code.tsv','w') as out:
    header  = fh.readline()
    for line in fh:
        line = line.strip()
        if line:
            lst = line.split(',')
#             print(lst)
            date = lst[1]
#             sample_date = datetime.strptime(date,'%Y%m%d')
            sample_date = datetime.now()
            code = lst[2]
            if code == 'ZSPDbs': #上海浦东国际机场里有ZSPD和ZSPDbs
                code = 'ZSPD'
            if code == 'nng': #南宁吴圩国际机场的原始数据里面有nng和NNG
                code = 'NNG'
            if code == 'zuck': #重庆
                code = 'ZUCK'                
            #
            gene = lst[3]
            path = lst[5]
            raw_data = ''
            with open(path) as p:
                raw_data = p.read()            

            # 
#             data = SeqData(path,gene,'F',sample_date,'')
            data = SeqData(path,gene,'F',sample_date,'')
            data.raw_data = raw_data
#             break

            channel = Channel.query.filter_by(code=code).first()
            if channel:
                if old_date == date: #相同则说明是同一天下单测序的：
                    if old_code == code : #相同则说明是同一个订单同一个机场（渠道）
                        order = old_order
                        order.seqData.append(data)
                    else:
                        randnum = str(random.randint(1000, 9999))
                        order_num = date+'_'+randnum
                        order = OrderSeq(order_num,sample_date)
                        db.session.add(order)
                else: # 不同一天，就建立新的订单：  
                    randnum = str(random.randint(1000, 9999))
                    order_num = date+'_'+randnum
                    order = OrderSeq(order_num,sample_date)

                u = User.query.filter_by(username = channel.name).first()
                if order not in u.orders_seq:
                    u.orders_seq.append(order)
                db.session.add(data)
            else:
                #找不到对应的编码的机场就记录下来：
                out.write(line+'\n')

            old_date = date
            old_code = code
            old_order = order
            

db.session.commit()   
#2300条测序记录保存到sqlite里面，sqlite的大小大概3.4M

In [ ]:
date = '20200427'

In [ ]:
datetime.now()

In [ ]:
datetime.strptime(date,'%Y%m%d')

p = 'bird_strike/20190729杭州萧山6个样品/测序结果/COI-2/良好/ZSHC-2019-64_COI-2_C1578.COI-2-F.PW908011904.seq'
with open(p) as o:
    raw = o.read()
len(raw)

with open('airport.csv') as fh,open('cannot_find_code.tsv','w') as out:
    header  = fh.readline()
    for line in fh:
        line = line.strip()
        if line:
            lst = line.split(',')
#             print(lst)
            date = lst[1]
#             sample_date = datetime.strptime(date,'%Y%m%d')
            sample_date = datetime.now()
            code = lst[2]
            gene = lst[3]
            path = lst[5]
            raw_data = ''
            with open(path) as p:
                raw_data = p.read()
                print(len(raw_data))

In [6]:
import re
file_name = 'ZBAD2020-300-yumao_COI-1_C2890.COI-1-bird-F.PW009290069.seq'
info = re.split(r'(COI-1|COI-2|L6615|H7956|cytb|L14995|H16065|BirdF1|BirdR1|Vercytb|L14841|H15149|ND2|L5219|H6313)',file_name)
print(info[0].strip('(fangdang)|(zuojiyi)|_|-|\.'))

ZBAD2020-300-yum


In [10]:
i = 'ZBAD2020-300-yumao'
i.rstrip('ymoau')

'ZBAD2020-300-'

In [12]:
i.endswith('ao')

True

In [ ]:
re.sub()